In [1]:
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

In [2]:
df = pd.read_csv('../data/decomp_pos.csv', index_col=0)

In [3]:
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [4]:
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [5]:
clf = rf()
clf = clf.fit(X_train, y_train)
rf_result = clf.predict_proba(X_test)
print(clf.score(X_test, y_test))

0.6815476190476191


In [6]:
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [7]:
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)
xgb_result = model.predict_proba(X_test)
print(model.score(X_test, y_test))

0.7886904761904762


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

lgb_result = gbm.predict_proba(X_test)
print(gbm.score(X_test, y_test))

[1]	valid_0's multi_logloss: 1.30385
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.23108
[3]	valid_0's multi_logloss: 1.17315
[4]	valid_0's multi_logloss: 1.12015
[5]	valid_0's multi_logloss: 1.07469
[6]	valid_0's multi_logloss: 1.03066
[7]	valid_0's multi_logloss: 0.991415
[8]	valid_0's multi_logloss: 0.956772
[9]	valid_0's multi_logloss: 0.922654
[10]	valid_0's multi_logloss: 0.896217
[11]	valid_0's multi_logloss: 0.870132
[12]	valid_0's multi_logloss: 0.843763
[13]	valid_0's multi_logloss: 0.821964
[14]	valid_0's multi_logloss: 0.805468
[15]	valid_0's multi_logloss: 0.786964
[16]	valid_0's multi_logloss: 0.77158
[17]	valid_0's multi_logloss: 0.753375
[18]	valid_0's multi_logloss: 0.738444
[19]	valid_0's multi_logloss: 0.725109
[20]	valid_0's multi_logloss: 0.713107
[21]	valid_0's multi_logloss: 0.702303
[22]	valid_0's multi_logloss: 0.692439
[23]	valid_0's multi_logloss: 0.683048
[24]	valid_0's multi_logloss: 0.671808
[25]	valid_0's mult

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
o = {}

In [10]:
o['xgb+rf'] = [round((sum(np.argmax((xgb_result+rf_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['xgb+lgb'] = [round((sum(np.argmax((xgb_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['rf+lgb'] = [round((sum(np.argmax((rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['rf+lgb+xgb'] = [round((sum(np.argmax((xgb_result+rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

In [11]:
pd.DataFrame(o).T

,0
xgb+rf,78.27
xgb+lgb,79.17
rf+lgb,77.08
rf+lgb+xgb,79.46


In [12]:
df = pd.read_csv('../data/decomp_neg.csv', index_col=0)

In [13]:
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [14]:
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [15]:
clf = rf()
clf = clf.fit(X_train, y_train)
rf_result = clf.predict_proba(X_test)
print(clf.score(X_test, y_test))

0.5957446808510638


In [16]:
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [17]:
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)
xgb_result = model.predict_proba(X_test)
print(model.score(X_test, y_test))

0.7446808510638298


/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

lgb_result = gbm.predict_proba(X_test)
print(gbm.score(X_test, y_test))

[1]	valid_0's multi_logloss: 1.29789
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.23416
[3]	valid_0's multi_logloss: 1.1783
[4]	valid_0's multi_logloss: 1.1247
[5]	valid_0's multi_logloss: 1.08614
[6]	valid_0's multi_logloss: 1.04632
[7]	valid_0's multi_logloss: 1.01259
[8]	valid_0's multi_logloss: 0.979809
[9]	valid_0's multi_logloss: 0.953402
[10]	valid_0's multi_logloss: 0.932058
[11]	valid_0's multi_logloss: 0.907093
[12]	valid_0's multi_logloss: 0.88651
[13]	valid_0's multi_logloss: 0.863204
[14]	valid_0's multi_logloss: 0.848956
[15]	valid_0's multi_logloss: 0.829107
[16]	valid_0's multi_logloss: 0.814619
[17]	valid_0's multi_logloss: 0.795882
[18]	valid_0's multi_logloss: 0.78049
[19]	valid_0's multi_logloss: 0.766839
[20]	valid_0's multi_logloss: 0.755909
[21]	valid_0's multi_logloss: 0.74466
[22]	valid_0's multi_logloss: 0.732723
[23]	valid_0's multi_logloss: 0.725083
[24]	valid_0's multi_logloss: 0.714402
[25]	valid_0's multi_log

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
o = {}

In [20]:
o['xgb+rf'] = [round((sum(np.argmax((xgb_result+rf_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['xgb+lgb'] = [round((sum(np.argmax((xgb_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['rf+lgb'] = [round((sum(np.argmax((rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]
o['rf+lgb+xgb'] = [round((sum(np.argmax((xgb_result+rf_result+lgb_result) , axis=1) == y_test)/len(y_test))*100, 2)]

In [21]:
pd.DataFrame(o).T

,0
xgb+rf,71.28
xgb+lgb,70.21
rf+lgb,71.28
rf+lgb+xgb,72.34
